In [6]:
import os

import dotenv
from pymongo import MongoClient
# from bson import ObjectId
import bson
import datetime

dotenv.load_dotenv()

SERVER_MONGO_URI = os.getenv("MONGO_URI_SERVER")

client = MongoClient(SERVER_MONGO_URI)

In [13]:
pipeline = [
    {'$match': {'createdAt': {'$gte': datetime.datetime(2025, 6, 8, 11, 7, 6, 429367), '$lte': datetime.datetime(2025, 7, 8, 11, 7, 6, 429405)}}}, {'$addFields': {'dateOnly': {'$dateToString': {'format': '%d/%m', 'date': '$createdAt'}}}}, {'$group': {'_id': '$dateOnly', 'Custo': {'$sum': {'$divide': ['$tokenValue', -1000000]}}, 'Mensagens': {'$sum': 1}, 'date_sort': {'$first': '$createdAt'}}}, {'$project': {'_id': 0, 'date': '$_id', 'Custo': {'$round': ['$Custo', 2]}, 'Mensagens': 1, 'date_sort': 1}}, {'$sort': {'date_sort': 1}}, {'$project': {'date': 1, 'Custo': 1, 'Mensagens': 1}}
]

result = list(client.LibreChat.transactions.aggregate(pipeline))
print(result)


[{'Mensagens': 263, 'date': '09/06', 'Custo': 0.37}, {'Mensagens': 298, 'date': '10/06', 'Custo': 1.85}, {'Mensagens': 59, 'date': '11/06', 'Custo': 0.21}, {'Mensagens': 121, 'date': '12/06', 'Custo': 0.15}, {'Mensagens': 39, 'date': '13/06', 'Custo': 0.06}, {'Mensagens': 8, 'date': '15/06', 'Custo': 0.04}, {'Mensagens': 62, 'date': '16/06', 'Custo': 0.09}, {'Mensagens': 280, 'date': '17/06', 'Custo': 2.22}, {'Mensagens': 124, 'date': '18/06', 'Custo': 0.84}, {'Mensagens': 6, 'date': '20/06', 'Custo': 0.04}, {'Mensagens': 83, 'date': '23/06', 'Custo': 0.18}, {'Mensagens': 106, 'date': '24/06', 'Custo': 0.19}, {'Mensagens': 161, 'date': '25/06', 'Custo': 0.32}, {'Mensagens': 336, 'date': '26/06', 'Custo': 2.31}, {'Mensagens': 143, 'date': '27/06', 'Custo': 2.89}, {'Mensagens': 184, 'date': '30/06', 'Custo': 1.69}, {'Mensagens': 109, 'date': '01/07', 'Custo': 0.8}, {'Mensagens': 50, 'date': '02/07', 'Custo': 0.1}, {'Mensagens': 207, 'date': '03/07', 'Custo': 0.55}, {'Mensagens': 219, 'da

# AUXILIARS FUNCTONS

In [3]:
def get_user_data(search_term, search_by='name'):
    if search_by == 'name':
        user = client.LibreChat.users.find_one({"name": search_term})
    else:
        user = client.LibreChat.users.find_one({"username": search_term})
    if not user:
        return "Usuário não encontrado"
    return user

get_user_data("Rafael Da Silva Melo")
get_user_data("rm810774", search_by='username')

{'_id': ObjectId('684818fd5e3aeef6d69ff1e9'),
 'name': 'Rafael Da Silva Melo',
 'username': 'rm810774',
 'email': 'rafaelmelo@hpeautos.com.br',
 'emailVerified': True,
 'provider': 'ldap',
 'role': 'USER',
 'group': 'default',
 'ldapId': 'rm810774',
 'plugins': [],
 'twoFactorEnabled': False,
 'termsAccepted': True,
 'backupCodes': [],
 'refreshToken': [],
 'createdAt': datetime.datetime(2025, 6, 10, 11, 37, 33, 520000),
 'updatedAt': datetime.datetime(2025, 7, 4, 11, 8, 4, 379000),
 '__v': 0}

# 1 - CRÉDITOS DE USUÁRIOS

## GERAL CREDITS

In [20]:
def geral_credits():
    geral_credits = []
    all_balances = list(client.LibreChat.balances.find())

    for balance in all_balances:
        user_obj = balance["user"]
        user_matricula = client.LibreChat.users.find_one({"_id": user_obj})["username"]
        user_name = client.LibreChat.users.find_one({"_id": user_obj})["name"]
        token_credits = balance["tokenCredits"]
        geral_credits.append([user_matricula,user_name, token_credits])

    return geral_credits

geral_credits()

[['gb810437', 'Gustavo Casadei Bellanda', 1000000],
 ['cs809362', 'Carlos H. P. dos Santos', 1000000],
 ['vs808910', 'Victor Hugo S. Calaca da Silva', 1000000],
 ['ms809859', 'Mateus Bonfim de Souza', 1000000],
 ['jc800888', 'Jeison Caldas', 1000000],
 ['cs811125', 'Carlos Daniel De Moura Santos', 1000000],
 ['cr804913', 'Claudia de Paula Rodrigues', 1000000],
 ['lr810746', 'Luis P. A. Rodrigues', 1000000],
 ['wl808976', 'Washington Machado Lourenco', 1000000],
 ['mc809289', 'Marco Antonio Cardoso Clares', 1000000],
 ['lp808381', 'Leonardo A. G. das N. de Paula', 1000000],
 ['ll010333', 'Luce Lopes', 1000000],
 ['ps810858', 'Pedro Augusto de A. N. Santana', 1000000],
 ['dm809144', 'Dhalton de Paula Marcelino', 1000000],
 ['ez800461', 'Eduardo Mauricio Zalamena', 98718534.04999998],
 ['gn809098', 'Gustavo Vilarinho Neto', 1000000],
 ['la802337', 'Lucas Bernardes de Assunção', 1000000],
 ['js811126', 'Joao Pedro Vales de Sousa', 1000000],
 ['gs810692', 'Guilherme De Oliveira Silva', 1000

## USER CREDITS

In [21]:
def get_user_credits(search_term, search_by='name'):
    try:
        user = get_user_data(search_term, search_by)
        balance = client.LibreChat.balances.find_one({"user": user["_id"]})
        if not balance:
            return (user["name"], 0)  # Retorna 0 se não tiver balanço
        return (user["name"], balance["tokenCredits"])
        
    except Exception as e:
        print(f"Erro ao buscar usuário: {e}")
        return None

# Buscar por NOME
result = get_user_credits("Rafael Da Silva Melo")
if result:
    nome, creditos = result
    print(f"Usuário: {nome}")
    print(f"Créditos: {creditos}")
else:
    print("Usuário não encontrado")

# Buscar por MATRICULA
result = get_user_credits("gb810437", search_by='username')
if result:
    nome, creditos = result
    print(f"Usuário: {nome}")
    print(f"Créditos: {creditos}")
else:
    print("Usuário não encontrado")


Usuário: Rafael Da Silva Melo
Créditos: 984681.3999999999
Usuário: Gustavo Casadei Bellanda
Créditos: 1000000


## TOP 10 MENORES CRÉDITOS

In [22]:
all_credits = geral_credits()

# TOP 10 MENORES CREDITOS
all_credits.sort(key=lambda x: x[2])
for credit in all_credits[:10]:
    print(f"{credit[0]} - {credit[1]} - {credit[2]}")


rm810774 - Rafael Da Silva Melo - 984681.3999999999
gb810437 - Gustavo Casadei Bellanda - 1000000
cs809362 - Carlos H. P. dos Santos - 1000000
vs808910 - Victor Hugo S. Calaca da Silva - 1000000
ms809859 - Mateus Bonfim de Souza - 1000000
jc800888 - Jeison Caldas - 1000000
cs811125 - Carlos Daniel De Moura Santos - 1000000
cr804913 - Claudia de Paula Rodrigues - 1000000
lr810746 - Luis P. A. Rodrigues - 1000000
wl808976 - Washington Machado Lourenco - 1000000


# 2 - ARQUIVOS

## GET_USER_ARCHIVES

In [52]:
# BUSCA ARQUIVOS DO USUÁRIO E RETORNA UMA LISTA COM OS DADOS FORMATADOS
def get_user_archives(search_term, search_by="name"):
    user = get_user_data(search_term, search_by)
    user_id = ObjectId(user["_id"])
    
    # Busca os arquivos do usuário
    arquivos_mongo = list(client.LibreChat.files.find({"user": user_id}))
    
    # Cria nova lista com os dados formatados
    arquivos_formatados = []
    for arquivo in arquivos_mongo:
        arquivos_formatados.append([
            arquivo["filename"],
            arquivo["type"],
            arquivo["bytes"] / (1024 * 1024)  # Convertendo para MB
        ])
    
    return arquivos_formatados

# IMPRIME OS ARQUIVOS DO USUÁRIO EM FORMA DE LISTA
def print_user_archives(user_name, search_by='name'):
    # Busca os arquivos
    arquivos = get_user_archives(user_name, search_by)

    # Imprime em formato de lista
    print(f"\nArquivos do usuário:")
    print("-" * 50)
    for i, arquivo in enumerate(arquivos, 1):
        print(f"Arquivo {i}:")
        print(f"Nome: {arquivo[0]}")
        print(f"Tipo: {arquivo[1]}")
        print(f"Tamanho: {arquivo[2]:.2f} MB")
        print("-" * 50)


print_user_archives("rm810774", search_by='username')




Arquivos do usuário:
--------------------------------------------------
Arquivo 1:
Nome: carta CNPJ.pdf
Tipo: application/pdf
Tamanho: 0.10 MB
--------------------------------------------------
Arquivo 2:
Nome: Automacao Nota de Debito email parabenização.pdf
Tipo: application/pdf
Tamanho: 0.40 MB
--------------------------------------------------
Arquivo 3:
Nome: MENU_9623.pdf
Tipo: application/pdf
Tamanho: 1.16 MB
--------------------------------------------------


## GET ALL USER ARCHIVES

In [62]:
def get_all_users_archives():
    all_users = list(client.LibreChat.users.find())
    all_users_archives = []
    for user in all_users:
        user_id = ObjectId(user["_id"])
        arquivos = list(client.LibreChat.files.find({"user": user_id}))
        # Adicionando a quantidade de arquivos na lista
        all_users_archives.append([
            user["username"], 
            user["name"], 
            arquivos,
            len(arquivos)  # Quantidade de arquivos
        ])
    return all_users_archives



def print_all_users_archives():
    # ORDERNAR POR NOME
    all_users_archives = get_all_users_archives()
    all_users_archives.sort(key=lambda x: x[1])

    # PRINTAR EM FORMATO DE LISTA
    for user in all_users_archives:
        username, name, arquivos, qtd = user  # Desempacotando os valores
        print(f"\nUsuário: {username} - {name}")
        print(f"Quantidade de arquivos: {qtd}")
        if qtd > 0:
            print("\nArquivos:")
            for arquivo in arquivos:
                print(f"- {arquivo['filename']} - {arquivo['type']} - {arquivo['bytes'] / (1024 * 1024):.2f} MB")
        print("-" * 50)



print_all_users_archives()


Usuário: aj810572 - Antonio Marmo Rodrigues Junior
Quantidade de arquivos: 4

Arquivos:
- clipboard_1749574963358_image.png - image/png - 0.13 MB
- clipboard_1751021443283_image.png - image/png - 0.03 MB
- clipboard_1751027189655_image.png - image/png - 0.00 MB
- codigo.txt - text/plain - 0.05 MB
--------------------------------------------------

Usuário: ba810651 - Barbara A de Almeida
Quantidade de arquivos: 0
--------------------------------------------------

Usuário: bm809292 - Bruno Borges Monteiro
Quantidade de arquivos: 0
--------------------------------------------------

Usuário: br811049 - Bruno Machado Ribeiro
Quantidade de arquivos: 3

Arquivos:
- imagem (6).png - image/png - 0.02 MB
- Chamados.xlsx - application/vnd.openxmlformats-officedocument.spreadsheetml.sheet - 0.14 MB
- Chamados.xlsx - application/vnd.openxmlformats-officedocument.spreadsheetml.sheet - 0.14 MB
--------------------------------------------------

Usuário: cs811125 - Carlos Daniel De Moura Santos
Qu

## TOP 10

In [ ]:

# TOP 10 PESSOAS QUE MAIS ENVIAM
def top_10_archives_senders():
    all_users_archives = get_all_users_archives()
    all_users_archives.sort(key=lambda x: x[3], reverse=True)
    return all_users_archives[:10]

# print(top_10_archives_senders())

# -------------------------------------------------------------------------

# PRINTAR TOP 10 PESSOAS QUE MAIS ENVIAM
def print_top_10_archives_senders():
    for user in top_10_archives_senders():
        print(f"Usuário: {user[0]} - {user[1]} - {user[3]}")

# print_top_10_archives_senders()

# -------------------------------------------------------------------------


# TOP 10 MAIORES ARQUIVOS
def top_10_biggest_archives():
    all_users_archives = get_all_users_archives()
    biggest_files = []
    
    # Percorre todos os usuários e seus arquivos
    for user in all_users_archives:
        username, name, files, _ = user
        
        # Percorre os arquivos do usuário
        for file in files:
            # Converte bytes para MB
            size_in_mb = file['bytes'] / (1024 * 1024)
            
            # Adiciona na lista com as informações relevantes
            biggest_files.append({
                'username': username,
                'user_name': name,
                'filename': file['filename'],
                'size_mb': size_in_mb,
                'type': file['type']
            })
    
    # Ordena por tamanho (maior para menor) e pega os 10 primeiros
    biggest_files.sort(key=lambda x: x['size_mb'], reverse=True)
    return biggest_files[:10]

# print(top_10_biggest_archives())


# PRINTAR TOP 10 MAIORES ARQUIVOS
def print_top_10_biggest_archives():
    files = top_10_biggest_archives()
    print("\nTop 10 maiores arquivos:")
    print("-" * 100)
    for i, file in enumerate(files, 1):
        print(f"{i}. Arquivo: {file['filename']}")
        print(f"   Usuário: {file['username']} - {file['user_name']}")
        print(f"   Tamanho: {file['size_mb']:.2f} MB")
        print(f"   Tipo: {file['type']}")
        print("-" * 100)

print_top_10_biggest_archives()






Top 10 maiores arquivos:
----------------------------------------------------------------------------------------------------
1. Arquivo: Kick Off - Projeto de Transformação Cultural em Saúde e Segurança.pdf
   Usuário: ll010333 - Luce Lopes
   Tamanho: 18.89 MB
   Tipo: application/pdf
----------------------------------------------------------------------------------------------------
2. Arquivo: Certidao Inteiro teor Fazenda Pires.pdf
   Usuário: ez800461 - Eduardo Mauricio Zalamena
   Tamanho: 3.90 MB
   Tipo: application/pdf
----------------------------------------------------------------------------------------------------
3. Arquivo: Certidao Inteiro teor Fazenda Pires.pdf
   Usuário: ez800461 - Eduardo Mauricio Zalamena
   Tamanho: 3.90 MB
   Tipo: application/pdf
----------------------------------------------------------------------------------------------------
4. Arquivo: Certidao Inteiro teor Fazenda Pires.pdf
   Usuário: ez800461 - Eduardo Mauricio Zalamena
   Tamanho: 3.9

# 3 - MENSAGENS

## USO GERAL

In [ ]:
# USO GERAL 

# USO GERAL POR PERÍODO
def get_all_users_messages(start_date=None, end_date=None, mensagens=True):
    """
    Retorna uma lista com o nome, quantidade de mensagens e (opcionalmente) as mensagens de todos os usuários,
    podendo filtrar por período.
    """
    all_users = list(client.LibreChat.users.find())
    all_users_messages = []

    # Prepara o filtro de data se fornecido
    date_filter = {}
    if start_date and end_date:
        start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
        date_filter = {"createdAt": {"$gte": start, "$lte": end}}

    for user in all_users:
        user_id_str = str(user["_id"])
        # Monta o filtro combinando usuário e, se houver, data
        user_filter = {"user": user_id_str}
        if date_filter:
            user_filter = {**user_filter, **date_filter}

        # Busca as mensagens ou só conta, conforme o parâmetro
        if mensagens:
            user_msgs = list(client.LibreChat.messages.find(user_filter))
        else:
            user_msgs = []

        all_users_messages.append([
            user["username"],
            user["name"],
            user_msgs,
            len(user_msgs) if mensagens else client.LibreChat.messages.count_documents(user_filter)
        ])

    return all_users_messages



def print_all_users_messages(ordem="", start_date=None, end_date=None, mensagens=True):
    all_users_messages = get_all_users_messages(start_date, end_date, mensagens)

    # Se mensagens=False, a contagem está em user[3]
    # Se mensagens=True, a contagem está em len(user[2]), mas também já está em user[3] pela função anterior

    if ordem == "nome":
        all_users_messages.sort(key=lambda x: x[1])            # nome
    else:
        all_users_messages.sort(key=lambda x: x[3], reverse=True)  # quantidade

    qty_total_messages = 0
    for user in all_users_messages:
        username, name, _mensagens, qtd = user
        print(f"\nUsuário: {username} - {name}")
        print(f"Quantidade de mensagens: {qtd}")
        print("-" * 50)
        qty_total_messages += qtd
    print("-" * 50)
    print(f"\nTotal de mensagens: {qty_total_messages}")
   


def print_all_users_messages_by_period(period="last_month", mensagens=False):
    if period == "last_day":
        start_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        end_date = (datetime.datetime.now()).strftime("%Y-%m-%d")
        print_all_users_messages(start_date=start_date, end_date=end_date, mensagens = mensagens)
    elif period == "last_week":
        start_date = (datetime.datetime.now() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
        end_date = (datetime.datetime.now()).strftime("%Y-%m-%d")
        print_all_users_messages(start_date=start_date, end_date=end_date, mensagens = mensagens)
    elif period == "last_month":
        start_date = (datetime.datetime.now() - datetime.timedelta(days=30)).strftime("%Y-%m-%d")
        end_date = (datetime.datetime.now()).strftime("%Y-%m-%d")
        print_all_users_messages(start_date=start_date, end_date=end_date, mensagens = mensagens)
    else:
        print("Período inválido")



# PRINTAR RANK DE MENSAGENS E SUAS QUANTIDADES POR PERÍODO
# print_all_users_messages(start_date="2025-07-01", end_date="2025-07-04", mensagens=True)

# PRINTAR TODOS OS USUÁRIOS E SUAS MENSAGENS POR PERÍODO
# print_all_users_messages_by_period(period="last_day", mensagens=True) # 1 dia
# print_all_users_messages_by_period(period="last_week", mensagens=True) # 1 semana
# print_all_users_messages_by_period(period="last_month", mensagens=True) # 1 mês




Usuário: ez800461 - Eduardo Mauricio Zalamena
Quantidade de mensagens: 464
--------------------------------------------------

Usuário: aj810572 - Antonio Marmo Rodrigues Junior
Quantidade de mensagens: 176
--------------------------------------------------

Usuário: rm810774 - Rafael Da Silva Melo
Quantidade de mensagens: 74
--------------------------------------------------

Usuário: lb810727 - Lorena da Silva Borges
Quantidade de mensagens: 53
--------------------------------------------------

Usuário: po809158 - Paula L Ribeiro de Oliveira
Quantidade de mensagens: 44
--------------------------------------------------

Usuário: cr804913 - Claudia de Paula Rodrigues
Quantidade de mensagens: 32
--------------------------------------------------

Usuário: lr810746 - Luis P. A. Rodrigues
Quantidade de mensagens: 32
--------------------------------------------------

Usuário: gs810692 - Guilherme De Oliveira Silva
Quantidade de mensagens: 32
--------------------------------------------

## TOP 10 MAIORES USERS COM QUANTIDADE DE MENSAGEM

In [30]:
#TOP 10 MAIORES USERS COM QUANTIDADES DE MENSAGENS

def print_top_10_biggest_users_messages(start_date=None, end_date=None, mensagens=True):

    all_users_messages = get_all_users_messages(start_date, end_date, mensagens)
    # Ordena pela quantidade de mensagens (user[3])
    all_users_messages.sort(key=lambda x: x[3], reverse=True)
    top_10 = all_users_messages[:10]

    print("TOP 10 USUÁRIOS COM MAIS MENSAGENS\n" + "="*40)
    for idx, user in enumerate(top_10, 1):
        username, name, _mensagens, qtd = user
        print(f"{idx:2d}. Usuário: {username} - {name}")
        print(f"    Quantidade de mensagens: {qtd}")
        print("-" * 40)


print_top_10_biggest_users_messages(start_date="2025-07-01", end_date="2025-07-04", mensagens=True)

TOP 10 USUÁRIOS COM MAIS MENSAGENS
 1. Usuário: ez800461 - Eduardo Mauricio Zalamena
    Quantidade de mensagens: 54
----------------------------------------
 2. Usuário: rj070253 - Roni Ribeiro de Jesus
    Quantidade de mensagens: 28
----------------------------------------
 3. Usuário: bm809292 - Bruno Borges Monteiro
    Quantidade de mensagens: 22
----------------------------------------
 4. Usuário: ps810858 - Pedro Augusto de A. N. Santana
    Quantidade de mensagens: 18
----------------------------------------
 5. Usuário: jm800945 - Juliano Martins P. Mitsutake
    Quantidade de mensagens: 10
----------------------------------------
 6. Usuário: rm810774 - Rafael Da Silva Melo
    Quantidade de mensagens: 8
----------------------------------------
 7. Usuário: fb811130 - Fabio Antonio Ferreira Barbosa
    Quantidade de mensagens: 6
----------------------------------------
 8. Usuário: lb810727 - Lorena da Silva Borges
    Quantidade de mensagens: 6
----------------------------

# 4 - CUSTO POR MENSAGENS (PERÍODO)

In [113]:

def get_costs_by_user_and_model(user=None, search_by="username", start_date=None, end_date=None):
    pipeline = []
    match = {}

    # Filtro por usuário
    if user:
        user_data = get_user_data(user, search_by)
        if not user_data or isinstance(user_data, str):
            print("Usuário não encontrado")
            return []
        match['user'] = user_data['_id']
        user_name = user_data['username']
    else:
        user_name = "Todos"

    # Filtro por período
    if start_date or end_date:
        match['createdAt'] = {}
        if start_date:
            match['createdAt']['$gte'] = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        if end_date:
            match['createdAt']['$lte'] = datetime.datetime.strptime(end_date, "%Y-%m-%d")
        # Remove o filtro se não foi preenchido corretamente
        if not match['createdAt']:
            del match['createdAt']
    else:
        start_date = datetime.datetime(2001, 1, 1)
        end_date = datetime.datetime.now()
        match['createdAt'] = {}
        match['createdAt']['$gte'] = datetime.datetime(2001, 1, 1)
        match['createdAt']['$lte'] = datetime.datetime.now()


    if match:
        pipeline.append({'$match': match})

    if user:
        pipeline.append({
            '$group': {
                '_id': '$user',
                'totalTokenValue': {'$sum': '$tokenValue'},
                'registros': {'$sum': 1}
            }
        })
    else:
        pipeline.append({
            '$group': {
                '_id': None,
                'totalTokenValue': {'$sum': '$tokenValue'},
                'registros': {'$sum': 1}
            }
        })

    pipeline.append({
        '$project': {
            'totalSpent': {'$divide': ['$totalTokenValue', -1_000_000]},
            'registros': 1
        }
    })
    pipeline.append({'$sort': {'custoTotal': -1}})

    result = list(client.LibreChat.transactions.aggregate(pipeline))
    # print("result", result)

    if user and result:
        for r in result:
            r['matricula'] = user_name

    return result

def print_costs_by_user_and_model(user=None, search_by="username", start_date=None, end_date=None):
    res = get_costs_by_user_and_model(user=user, search_by=search_by, start_date=start_date, end_date=end_date)
    for r in res:
        print(f"{r.get('matricula', 'Todos')}: {r['totalSpent']:.3f} USD total gasto em {r['registros']} transactions")



print_costs_by_user_and_model(user="rm810774", start_date="2025-06-01", end_date="2025-07-01")
print_costs_by_user_and_model(user="gb810437", start_date="2025-06-01", end_date="2025-07-01")
print_costs_by_user_and_model(start_date="2025-06-01", end_date="2025-07-01")




rm810774: 0.638 USD total gasto em 97 transactions
gb810437: 1.910 USD total gasto em 1011 transactions
Todos: 14.626 USD total gasto em 2718 transactions


## TOP 10 USERS

In [115]:
# TOP 10 USUÁRIOS MAIS GASTADORES
def get_top_10_users_by_spent(start_date=None, end_date=None):
    # Converte datas para datetime, se forem string
    if isinstance(start_date, str):
        start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    if isinstance(end_date, str):
        end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

    
    pipeline = [
        {
            '$match': {
                'createdAt': {
                    '$gte': start_date,
                    '$lte': end_date
                }
            }
        },
        {
            '$group': {
                '_id': '$user',
                'totalTokenValue': {'$sum': '$tokenValue'},
                'registros': {'$sum': 1}
            }
        },
        {
            '$project': {
                'totalSpent': {'$divide': ['$totalTokenValue', -1_000_000]},
                'registros': 1
            }
        },
        {
            '$sort': {'totalSpent': -1}
        },
        {
            '$limit': 10
        },
        {
            '$lookup': {
                'from': 'users',
                'localField': '_id',
                'foreignField': '_id',
                'as': 'user_info'
            }
        },
        {
            '$unwind': '$user_info'
        },
        {
            '$project': {
                'username': '$user_info.username',
                'name': '$user_info.name',
                'totalSpent': 1,
                'registros': 1
            }
        }
    ]
    return list(client.LibreChat.transactions.aggregate(pipeline))



def print_top_10_users_by_spent(start_date=None, end_date=None):
    res = get_top_10_users_by_spent(start_date=start_date, end_date=end_date)
    print("TOP 10 USUÁRIOS MAIS GASTADORES")
    for idx, r in enumerate(res, 1):
        print(f"{idx:2d}. {r['username']} - {r['name']}: {r['totalSpent']:.3f} USD em {r['registros']} transactions")

print_top_10_users_by_spent(start_date="2020-01-01", end_date="2025-07-31")

TOP 10 USUÁRIOS MAIS GASTADORES
 1. ez800461 - Eduardo Mauricio Zalamena: 4.631 USD em 584 transactions
 2. ps810858 - Pedro Augusto de A. N. Santana: 4.251 USD em 434 transactions
 3. aj810572 - Antonio Marmo Rodrigues Junior: 2.680 USD em 253 transactions
 4. gb810437 - Gustavo Casadei Bellanda: 2.013 USD em 1193 transactions
 5. lb810727 - Lorena da Silva Borges: 1.073 USD em 68 transactions
 6. rm810774 - Rafael Da Silva Melo: 0.757 USD em 124 transactions
 7. as810618 - Antony Marcolino Silva: 0.440 USD em 45 transactions
 8. rj070253 - Roni Ribeiro de Jesus: 0.157 USD em 40 transactions
 9. pj810185 - Paulo Jose Oliveira Junior: 0.133 USD em 26 transactions
10. ba810651 - Barbara A de Almeida: 0.127 USD em 32 transactions


# 5 - MODELOS


## USO GERAL

In [37]:
# MODELOS MAIS USADOS

def get_models_usage():
    pipeline = [
        {"$group": {"_id": "$model", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    return list(client.LibreChat.conversations.aggregate(pipeline))

def get_deleted_messages_models_usage():
    pipeline = [
        {"$group": {"_id": "$model", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    return list(client.LibreChat.conversationhistories.aggregate(pipeline))

from collections import defaultdict

def somar_modelos(*listas):
    soma = defaultdict(int)
    for lista in listas:
        for item in lista:
            modelo = item['_id']
            soma[modelo] += item['count']
    # Converte para lista de dicionários e ordena
    resultado = [{'_id': modelo, 'count': count} for modelo, count in soma.items()]
    resultado.sort(key=lambda x: x['count'], reverse=True)
    return resultado


resultado_final = somar_modelos(get_models_usage(), get_deleted_messages_models_usage())


def print_top_10_models(models):
    print("\nTOP 10 MAIS USADOS")
    print("=" * 50)
    print(models)
    for idx, item in enumerate(models[:10], 1):
        print(f"{idx:2d}. Modelo: {item['_id']}")
        print(f"    Quantidade: {item['count']}")
        print("-" * 50)


print_top_10_models(resultado_final)


TOP 10 MAIS USADOS
[{'_id': 'gemini-2.5-flash-preview-05-20', 'count': 89}, {'_id': 'gemini-2.5-flash', 'count': 70}, {'_id': 'gpt-4.1-mini', 'count': 55}, {'_id': 'gpt-4.1', 'count': 48}, {'_id': 'gemini-2.5-flash-preview-04-17', 'count': 33}, {'_id': 'claude-sonnet-4-20250514', 'count': 31}, {'_id': 'o4-mini', 'count': 30}, {'_id': 'meta/llama-4-maverick-17b-128e-instruct', 'count': 27}, {'_id': 'gpt-4.1-2025-04-14', 'count': 24}, {'_id': 'gemini-2.5-pro', 'count': 24}, {'_id': None, 'count': 23}, {'_id': 'gemini-2.5-pro-preview-06-05', 'count': 21}, {'_id': 'deepseek-ai/deepseek-r1', 'count': 16}, {'_id': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'count': 16}, {'_id': 'gpt-4.1-mini-2025-04-14', 'count': 10}, {'_id': 'meta-llama/llama-4-scout-17b-16e-instruct', 'count': 9}, {'_id': 'gpt-4.1-nano-2025-04-14', 'count': 8}, {'_id': 'qwen/qwen3-235b-a22b', 'count': 7}, {'_id': 'chatgpt-4o-latest', 'count': 7}, {'_id': 'gemini-2.0-flash-lite', 'count': 6}, {'_id': 'nvidia/llama-3.

In [44]:
# USO POR USUÁRIO MODELO

def get_user_model_usage(user, search_by="username"):
    id = str(get_user_data(user, search_by =search_by)["_id"])

    pipeline = [
        {"$match": {"user": id}},
        {"$group": {"_id": "$model", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]
    return list(client.LibreChat.conversations.aggregate(pipeline))

# print(get_user_model_usage("rm810774"))

def print_user_model_usage(user, search_by="username"):
    for model in get_user_model_usage(user, search_by):
        print(f"{model['_id']}: {model['count']}")


print_user_model_usage("rm810774")


gemini-2.5-pro: 7
gpt-4.1: 2
o4-mini: 1
gemini-2.5-pro-preview-06-05: 1
gpt-4.1-2025-04-14: 1
claude-sonnet-4-20250514: 1
gemini-2.5-flash-preview-05-20: 1
